## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

Add our `utils` directory to the system's `$PATH` so we can import Python files from sibling directories.

In [1]:
import os
import pathlib

In [2]:
this_dir = pathlib.Path(os.path.abspath(""))

In [3]:
data_dir = this_dir / "data"

In [4]:
import pytz
import glob
import requests
import pandas as pd
from slugify import slugify
from datetime import datetime, date

/Users/slee/.local/share/virtualenvs/california-coronavirus-scrapers-QTClc1WQ/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Download

In [5]:
url = "https://services5.arcgis.com/ROBnTHSNjoZ2Wm1P/ArcGIS/rest/services/COVID_19_Case_Rates_by_Zip_Code_WM/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token="

In [6]:
r = requests.get(url)

In [7]:
data = r.json()

## Parse

In [8]:
values = []

In [9]:
for f in data["features"]:
    row = f["attributes"].values()
    values.append(row)

In [10]:
cols = list(data["features"][0]["attributes"].keys())

Convert to dataframe

In [11]:
df = pd.DataFrame(values, columns=cols)

Get timestamp

In [12]:
date_url = "https://services5.arcgis.com/ROBnTHSNjoZ2Wm1P/ArcGIS/rest/services/COVID_19_Case_Rates_by_Zip_Code_WM/FeatureServer/0/?f=json"

In [13]:
date_r = requests.get(date_url)

In [14]:
date_data = date_r.json()

In [15]:
timestamp = date_data["editingInfo"]["lastEditDate"]

In [16]:
timestamp = datetime.fromtimestamp((timestamp / 1000))

In [17]:
timestamp

datetime.datetime(2023, 2, 24, 3, 37, 30, 121000)

In [18]:
update_date = pd.to_datetime(timestamp).round(freq='S')

In [19]:
update_date

Timestamp('2023-02-24 03:37:30')

Add the timestamp to the dataframe

In [20]:
df["county_date"] = update_date

In [21]:
df

,Zip_Number,Zip_Alpha,Population,Cases,CaseRates,GlobalID,OBJECTID,Shape__Area,Shape__Length,county_date
0,94502,94502,14061,2107,14984.709480,2201afb0-f24a-4caa-a280-7b078e60745a,1,1.032795e+07,14675.181018,2023-02-24 03:37:30
1,94706,94706,20331,3027,14888.593773,17902952-3a86-466c-9d24-ac3111f571aa,2,5.654830e+06,13201.790210,2023-02-24 03:37:30
2,94709,94709,11734,1791,15263.337310,6385fe10-a831-42df-9145-4d33725a691a,3,2.347100e+06,7845.460234,2023-02-24 03:37:30
3,94710,94710,8458,2128,25159.612201,a968be73-65c9-4d4a-96ee-825e20a407c0,4,1.132117e+07,34031.634584,2023-02-24 03:37:30
4,94587,94587,72559,17028,23467.798619,7da97736-e2af-4136-8312-842821133991,5,1.200952e+08,86993.659936,2023-02-24 03:37:30
5,94577,94577,48690,12212,25081.125488,4e279009-0fd3-405a-afca-2b1f9c2ecf35,6,3.364755e+07,48564.165519,2023-02-24 03:37:30
6,94605,94605,42945,11246,26186.983351,778ec18a-f365-4942-af43-6a8377e3834e,7,3.629375e+07,36745.169765,2023-02-24 03:37:30
7,94606,94606,37481,8953,23886.769296,79272129-630e-4d06-9fdf-0956e7fa8b7d,8,9.712934e+06,20368.808704,2023-02-24 03:37:30
8,94608,94608,32602,7283,22339.120299,803f9089-a635-4eb0-b1b8-00db378a418b,9,1.152262e+07,22652.244258,2023-02-24 03:37:30
9,94610,94610,30892,5977,19348.051275,ee76320b-b507-407b-88fb-cdde5582965c,10,8.594601e+06,21468.664491,2023-02-24 03:37:30


Rename and trim

In [22]:
clean_df = df[["Zip_Alpha", "Cases", "county_date"]].rename(
    columns={"Zip_Alpha": "zip", "Cases": "confirmed_cases"}
)

Match city names to zips

In [23]:
zips = [
    "94502",
    "94577",
    "94579",
    "94603",
    "94605",
    "94621",
    "94501",
    "94606",
    "94607",
    "94610",
    "94612",
    "94608",
    "94609",
    "94618",
    "94702",
    "94703",
    "94704",
    "94705",
    "94709",
    "94710",
    "94720",
    "94601",
    "94602",
    "94611",
    "94613",
    "94619",
    "94706",
    "94707",
    "94708",
    "94538",
    "94555",
    "94560",
    "94539",
    "94550",
    "94536",
    "94542",
    "94544",
    "94545",
    "94587",
    "94541",
    "94546",
    "94552",
    "94578",
    "94580",
    "94566",
    "94568",
    "94586",
    "94588",
    "94551",
    "95391",
    "94514",
    "94505",
    "95377",
]

In [24]:
cities = [
    "Alameda",
    "San Leandro",
    "San Leandro",
    "Oakland",
    "Oakland",
    "Oakland",
    "Alameda",
    "Oakland",
    "Oakland",
    "Oakland",
    "Oakland",
    "Emeryville",
    "Oakland",
    "Oakland",
    "Berkeley",
    "Berkeley",
    "Berkeley",
    "Berkeley",
    "Berkeley",
    "Berkeley",
    "Berkeley",
    "Oakland",
    "Oakland",
    "Oakland",
    "Oakland",
    "Oakland",
    "Albany",
    "Berkeley",
    "Berkeley",
    "Fremont",
    "Fremont",
    "Newark",
    "Fremont",
    "Livermore",
    "Fremont",
    "Hayward",
    "Hayward",
    "Hayward",
    "Union City",
    "Hayward",
    "Castro Valley",
    "Castro Valley",
    "San Leandro",
    "San Lorenzo",
    "Pleasanton",
    "Dublin",
    "Sunol",
    "Pleasanton",
    "Livermore",
    "Mountain House",
    "Byron",
    "Discovery Bay",
    "Tracy",
]

In [25]:
d = {"zip": zips, "city": cities}

In [26]:
matchup_df = pd.DataFrame(d)

In [27]:
merge_df = pd.merge(clean_df, matchup_df, how="left", on="zip")

In [28]:
merge_df["area"] = merge_df["zip"] + ": " + merge_df["city"]

Clean up

In [29]:
merge_df["county"] = "Alameda"

In [30]:
merge_df = merge_df[["county", "area", "county_date", "confirmed_cases"]]

Drop mysterious blank rows

In [31]:
filter_df = merge_df.dropna(subset=["area"])

Fill empty cases with 0

In [32]:
filter_df["confirmed_cases"] = filter_df.confirmed_cases.fillna(0)

Filter down to the latest date

In [33]:
export_df = filter_df[filter_df.county_date == filter_df.county_date.max()]

## Vet

In [34]:
try:
    assert not len(export_df) > 53
except AssertionError:
    raise AssertionError("Alameda scraper has additional rows than before")

In [35]:
try:
    assert not len(export_df) < 53
except AssertionError:
    raise AssertionError("Alameda County scraper is missing rows")

In [36]:
export_df

,county,area,county_date,confirmed_cases
0,Alameda,94502: Alameda,2023-02-24 03:37:30,2107
1,Alameda,94706: Albany,2023-02-24 03:37:30,3027
2,Alameda,94709: Berkeley,2023-02-24 03:37:30,1791
3,Alameda,94710: Berkeley,2023-02-24 03:37:30,2128
4,Alameda,94587: Union City,2023-02-24 03:37:30,17028
5,Alameda,94577: San Leandro,2023-02-24 03:37:30,12212
6,Alameda,94605: Oakland,2023-02-24 03:37:30,11246
7,Alameda,94606: Oakland,2023-02-24 03:37:30,8953
8,Alameda,94608: Emeryville,2023-02-24 03:37:30,7283
9,Alameda,94610: Oakland,2023-02-24 03:37:30,5977


## Export

Mark the current date

In [37]:
tz = pytz.timezone("America/Los_Angeles")

In [38]:
today = datetime.now(tz).date()

In [39]:
slug = slugify("Alameda")

In [40]:
export_df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [41]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [42]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [43]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [44]:
df.to_csv(os.path.join(data_dir / slug / "timeseries.csv"), index=False)

In [48]:
df.tail(20)

,county,area,county_date,confirmed_cases,date
20,Alameda,94609: Oakland,2023-02-24 03:37:30,5381,2023-02-24
9,Alameda,94610: Oakland,2023-02-24 03:37:30,5977,2023-02-24
25,Alameda,94611: Oakland,2023-02-24 03:37:30,6821,2023-02-24
21,Alameda,94612: Oakland,2023-02-24 03:37:30,4335,2023-02-24
49,Alameda,94613: Oakland,2023-02-24 03:37:30,8,2023-02-24
22,Alameda,94618: Oakland,2023-02-24 03:37:30,2396,2023-02-24
18,Alameda,94619: Oakland,2023-02-24 03:37:30,5476,2023-02-24
27,Alameda,94621: Oakland,2023-02-24 03:37:30,11520,2023-02-24
10,Alameda,94702: Berkeley,2023-02-24 03:37:30,3500,2023-02-24
11,Alameda,94703: Berkeley,2023-02-24 03:37:30,3943,2023-02-24


In [45]:
# df = pd.read_csv(os.path.join(data_dir / slug / "timeseries.csv"), parse_dates=["county_date"])
# df["date"] = pd.to_datetime(df.date, format= "%Y-%m-%d %H:%M:%S")
# df["date"] =  df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
# df.to_csv(os.path.join(data_dir / slug / "timeseries.csv"), index=False)